<a href="https://colab.research.google.com/github/Sayed-Hossein-Hosseini/Node_Classification_in_the_Amazon_Product_Graph/blob/master/Edge_Prediction_in_the_Amazon_Product_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Edge Prediction in the Amazon Product Graph**